In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Importing CRUD file
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# CRUD Python module name

username = "aacuser"
password = "SNHU1pass"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


# updates length of long double values
df['location_long'] = df['location_long'].apply(lambda x: round(x, 4))
df['location_lat'] = df['location_lat'].apply(lambda x: round(x, 4))
df['age_upon_outcome_in_weeks'] = df['age_upon_outcome_in_weeks'].apply(lambda x: round(x, 2))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'GS-Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    
    # Display logo and unique Identifier
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 html.A(href="https://www.snhu.edu",
                        children=[
                            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width='200', height='200')]
                       ),
                 # Unique identifier
                html.H2("Jose Lemus"),
                 
        ]),
    
    html.Hr(),
        # Table filter
        html.Div([
        html.Span("sort by"),
        dcc.Dropdown(
            id='rescue-dropdown',
            options=['Disaster Rescue', 'Mountain Rescue', 'Water Rescue', 'Reset'])
        ], style={'width':'200px'}),

    html.Hr(),

    # Interactive table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            # Updates field names to improve readability for the user
            {"name": "Number", "id": "rec_num", "deletable": False, "selectable": True},
            {"name": "Age", "id": "age_upon_outcome", "deletable": False, "selectable": True},
            {"name": "ID", "id": "animal_id", "deletable": False, "selectable": True},
            {"name": "Animal", "id": "animal_type", "deletable": False, "selectable": True},
            {"name": "Breed", "id": "breed", "deletable": False, "selectable": True},
            {"name": "Color", "id": "color", "deletable": False, "selectable": True},
            {"name": "Birth Date", "id": "date_of_birth", "deletable": False, "selectable": True},
            {"name": "Datetime", "id": "datetime", "deletable": False, "selectable": True},
            {"name": "Name", "id": "name", "deletable": False, "selectable": True},
            {"name": "Subtype", "id": "outcome_subtype", "deletable": False, "selectable": True},
            {"name": "Type", "id": "outcome_type", "deletable": False, "selectable": True},
            {"name": "Sex", "id": "sex_upon_outcome", "deletable": False, "selectable": True},
            {"name": "Latitude", "id": "location_lat", "deletable": False, "selectable": True},
            {"name": "Longitude", "id": "location_long", "deletable": False, "selectable": True},
            {"name": "Age in weeks", "id": "age_upon_outcome_in_weeks", "deletable": False, "selectable": True},
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        #sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10

    ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(Output('datatable-id', "data"),
             Input('rescue-dropdown', 'value'))
def on_click(rescueType):
    # base case
    df = pd.DataFrame.from_records(db.read({}))
    
    # evaluates rescueType case and generates a query ased on the result
    if rescueType == 'Water Rescue':
        # build query
        query = {"animal_type": "Dog",
                "breed": {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                }
        
    elif rescueType == 'Mountain Rescue':
        # build query
        query = {"animal_type": "Dog",
                "breed": 
                 {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                }
        
    elif rescueType == 'Disaster Rescue':
        # build query
        query = {"animal_type": "Dog",
                "breed": {"$in" : ["German Shepherd", "Doberman Pinscher", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
                }
        
    else:
        query = {}

    df = pd.DataFrame.from_records(db.read(query))
        
    # updates length of long double values
    df['location_long'] = df['location_long'].apply(lambda x: round(x, 4))
    df['location_lat'] = df['location_lat'].apply(lambda x: round(x, 4))
    df['age_upon_outcome_in_weeks'] = df['age_upon_outcome_in_weeks'].apply(lambda x: round(x, 2))
    
    # cleanup Mongo _id field
    df.drop(columns=['_id'], inplace=True)
    
    return df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):    
    
    # ensures that viewData is not empty
    if viewData is None:
        viewData = pd.DataFrame.from_records(db.read({}))
        
        
    # Displays pie chart based on viewData
    return [
        dcc.Graph(            
            figure = px.pie(viewData, names='breed', title='Preferred Animals'),
            style={'width': '500px', 'height': '500px'}
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    
# Adds in the geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
        
    if not dff.empty:
        return [dl.Map(style={'width': '900px', 'height': '500px'},
                  center=[30.75, -97.48], zoom=10, children=[
                      dl.TileLayer(id="base-layer-id"),
                      # Marker with tool tip and popup
                      # column 13 and 14 define the grid-coordinates for the map
                      # Column 4 defines the reed of the animal
                      # Column 9 defines the name of the animal
                        dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                            children=[
                                dl.Tooltip(dff.iloc[row,4]),
                                dl.Popup([
                                    html.H1("Animal Name"),
                                    html.P(dff.iloc[row,9])
                                   ])
                               ])
                      ])
               ]



app.run_server(debug=True)


Conneted to database
Dash app running on http://127.0.0.1:30069/
